In [1]:
import getPreProcessingFunction as PPM
import BiVEDA_Function as BiVEDA
import UniVEDA_catFunction as catUniVEDA
import UniVEDA_conti_methods as contiUniVEDA
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,roc_auc_score,roc_curve, precision_score,f1_score,auc,precision_recall_curve,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
%matplotlib inline

getFuncs()
getFuncs()
getFuncs()
getFuncs()


In [2]:
test=pd.read_csv("/Users/keeratjohar2305/Downloads/AV_ML_practice_Notebooks/test_JantaHack_Cabmobility.csv")
print(test.shape)
test.head()

(87395, 13)


,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender
0,T0005689459,9.44,A,10.0,2.57438,A,B,3.68000,2,NaN,46,63,Male
1,T0005689462,32.15,B,10.0,2.85143,A,A,1.59375,0,65.0,49,80,Female
2,T0005689463,10.38,C,4.0,2.70530,A,D,4.50500,0,NaN,47,74,Male
3,T0005689466,14.94,NaN,6.0,2.48159,C,E,4.53000,0,63.0,43,54,Male
4,T0005689468,32.03,B,7.0,2.81598,A,K,4.60125,3,96.0,44,56,Male


In [3]:
train=pd.read_csv("/Users/keeratjohar2305/Downloads/AV_ML_practice_Notebooks/train_JantaHack_Cabmobility.csv")
tr_copy=train.copy()
ntrain=train.shape[0]
print(ntrain,train.shape)
train.head()

131662 (131662, 14)


,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,B,1.0,2.42769,A,A,3.90500,0,40.0,46,60,Female,2
1,T0005689461,29.47,B,10.0,2.78245,B,A,3.45000,0,38.0,56,78,Male,2
2,T0005689464,41.58,NaN,10.0,NaN,NaN,E,3.50125,2,NaN,56,77,Male,2
3,T0005689465,61.56,C,10.0,NaN,NaN,A,3.45375,0,NaN,52,74,Male,3
4,T0005689467,54.95,C,10.0,3.03453,B,A,3.40250,4,51.0,49,102,Male,2


# Dynamic pricing 
also referred to as surge pricing, demand pricing, or time-based pricing is a pricing strategy in which businesses set flexible prices for products or services based on current market demands

Logistic regression isn't appropriate, because the dependent variable isn't binary. Multinomial logistic regression is applicable, but ignores the ordering (presumably class A is more fit to work than class B, which is more fit to work than class C, and so forth), and will require many more parameters, with a corresponding loss of efficiency.

# Multiple Class
Algorithm Choice: 
                RF,
                Decision tree, 
                naive bayes 
                SVM,
                But not logistic regression

In [4]:
label_col='Surge_Pricing_Type'

In [5]:
train[label_col].value_counts()/ntrain

2    0.430861
3    0.362443
1    0.206696
Name: Surge_Pricing_Type, dtype: float64

In [6]:

PPM.desc1(train,2,label_col)

,dtypes,null_count,total count,unique_count,missing value ratio,variance of numerics,std,mean,min,max,skew,kurt,Oridinals SpearmanCorr withTarget
Surge_Pricing_Type,int64,0.0,131662.0,3.0,0.00,0.54,0.74,2.16,1.00000,3.00,-0.255,-1.137,1.00000
Type_of_Cab,object,20210.0,111452.0,5.0,15.35,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.57134
Cancellation_Last_1Month,int64,0.0,131662.0,9.0,0.00,1.08,1.04,0.78,0.00000,8.00,1.551,2.700,0.19586
Customer_Rating,float64,0.0,131662.0,3931.0,0.00,0.96,0.98,2.85,0.00125,5.00,-0.191,-0.540,0.14932
Confidence_Life_Style_Index,object,20193.0,111469.0,3.0,15.34,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.12556
Trip_Distance,float64,0.0,131662.0,10326.0,0.00,651.42,25.52,44.20,0.31000,109.23,0.724,-0.146,0.12111
Life_Style_Index,float64,20193.0,111469.0,55978.0,15.34,0.05,0.23,2.80,1.59638,4.88,0.194,1.112,0.07855
Destination_Type,object,0.0,131662.0,14.0,0.00,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.04921
Var3,int64,0.0,131662.0,96.0,0.00,134.06,11.58,75.10,52.00000,206.00,0.829,1.034,0.04462
Var1,float64,71030.0,60632.0,122.0,53.95,476.13,21.82,64.20,30.00000,210.00,0.465,-0.711,0.04234


In [7]:
# append two training and test dataset
df=train.append(test)


In [8]:
# Lets using missing data
sammy=PPM.desc1(df,2,label_col)
sammy

,dtypes,null_count,total count,unique_count,missing value ratio,variance of numerics,std,mean,min,max,skew,kurt,Oridinals SpearmanCorr withTarget
Surge_Pricing_Type,float64,87395.0,131662.0,3.0,39.90,0.54,0.74,2.16,1.00000,3.00,-0.255,-1.137,1.00000
Type_of_Cab,object,33368.0,185689.0,5.0,15.23,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.57134
Cancellation_Last_1Month,int64,0.0,219057.0,9.0,0.00,1.07,1.04,0.78,0.00000,8.00,1.544,2.673,0.19586
Customer_Rating,float64,0.0,219057.0,3976.0,0.00,0.96,0.98,2.85,0.00125,5.00,-0.193,-0.537,0.14932
Confidence_Life_Style_Index,object,33520.0,185537.0,3.0,15.30,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.12556
Trip_Distance,float64,0.0,219057.0,10594.0,0.00,650.63,25.51,44.16,0.31000,109.23,0.726,-0.142,0.12111
Life_Style_Index,float64,33520.0,185537.0,69677.0,15.30,0.05,0.23,2.80,1.31785,4.88,0.206,1.137,0.07855
Destination_Type,object,0.0,219057.0,14.0,0.00,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.04921
Var3,int64,0.0,219057.0,102.0,0.00,134.10,11.58,75.07,52.00000,206.00,0.819,0.957,0.04462
Var1,float64,117819.0,101238.0,130.0,53.78,472.93,21.75,64.10,30.00000,210.00,0.467,-0.728,0.04234


In [9]:
# cat types cols
cat_colsdf=sammy[sammy.unique_count<15]
cat_colsdf

,dtypes,null_count,total count,unique_count,missing value ratio,variance of numerics,std,mean,min,max,skew,kurt,Oridinals SpearmanCorr withTarget
Surge_Pricing_Type,float64,87395.0,131662.0,3.0,39.90,0.54,0.74,2.16,1.0,3.0,-0.255,-1.137,1.00000
Type_of_Cab,object,33368.0,185689.0,5.0,15.23,0.00,0.00,0.00,0.0,0.0,0.000,0.000,0.57134
Cancellation_Last_1Month,int64,0.0,219057.0,9.0,0.00,1.07,1.04,0.78,0.0,8.0,1.544,2.673,0.19586
Confidence_Life_Style_Index,object,33520.0,185537.0,3.0,15.30,0.00,0.00,0.00,0.0,0.0,0.000,0.000,0.12556
Destination_Type,object,0.0,219057.0,14.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000,0.000,0.04921
Customer_Since_Months,float64,9886.0,209171.0,11.0,4.51,13.15,3.63,6.01,0.0,10.0,-0.242,-1.446,0.02789
Gender,object,0.0,219057.0,2.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000,0.000,0.00118


In [10]:
#catUniVEDA.BiVEDA_cat_cols_bars_over_ybinary(cat_colsdf.index[:8],label_col,df)

In [11]:
df['Type_of_Cab'].value_counts()

B    51585
C    46732
A    35878
D    31885
E    19609
Name: Type_of_Cab, dtype: int64

In [12]:
#np.ceil(df['Customer_Rating'])#.dtypes#.astype('int64')#.value_counts()

In [13]:
from decimal import localcontext, Decimal, ROUND_HALF_UP
#temp=pd.DataFrame()
#temp['Type_of_Cab']=df['Type_of_Cab']
#temp['Customer_Rating']=df['Customer_Rating']
df['CR']=df['Customer_Rating'].apply(lambda x: Decimal(x).to_integral_exact(rounding=ROUND_HALF_UP))
df['CR']=df['CR'].astype('int32')
#temp['CR'].value_counts()

In [14]:
df['Destination_Type'].value_counts()

A    129010
B     49193
C     12397
D     11085
E      4549
F      3222
G      2513
H      2124
I      1334
J      1166
K      1102
L      1052
M       160
N       150
Name: Destination_Type, dtype: int64

In [15]:
df.groupby(['Destination_Type'])['Trip_Distance'].agg(['count','mean'])

,count,mean
Destination_Type,,
A,129010,46.608951
B,49193,45.492173
C,12397,42.704003
D,11085,29.609426
E,4549,32.161677
F,3222,46.892992
G,2513,24.958949
H,2124,27.530127
I,1334,22.765735


In [16]:
df.groupby(['CR'])['Trip_Distance'].agg(['count','mean'])

,count,mean
CR,,
0,1281,42.680258
1,20652,44.319500
2,56449,45.492127
3,79845,45.329998
4,52984,42.114545
5,7846,36.268439


In [17]:
cat_colsdf.index

Index(['Surge_Pricing_Type', 'Type_of_Cab', 'Cancellation_Last_1Month',
       'Confidence_Life_Style_Index', 'Destination_Type',
       'Customer_Since_Months', 'Gender'],
      dtype='object')

In [18]:
PPM.Ch2Test_CatAssociation_over_yCat(cat_colsdf.index,'Type_of_Cab',df)

,label_col,colname,statistic,p_value,dof,expected_frequency
Type_of_Cab,Type_of_Cab,Type_of_Cab,39.000000,0.000,9.0,"[[1.9230769230769231, 1.5384615384615385, 0.76..."
Cancellation_Last_1Month,Type_of_Cab,Cancellation_Last_1Month,15.754762,0.072,9.0,"[[2.6923076923076925, 2.1538461538461537, 1.07..."
Customer_Since_Months,Type_of_Cab,Customer_Since_Months,13.714286,0.547,15.0,"[[0.3333333333333333, 0.3333333333333333, 0.16..."
Surge_Pricing_Type,Type_of_Cab,Surge_Pricing_Type,4.333333,0.632,6.0,"[[0.38461538461538464, 0.3076923076923077, 0.1..."
Gender,Type_of_Cab,Gender,3.423333,0.331,3.0,"[[1.1538461538461537, 0.9230769230769231, 0.46..."
Confidence_Life_Style_Index,Type_of_Cab,Confidence_Life_Style_Index,3.100000,0.796,6.0,"[[0.5, 0.2, 0.2, 0.1], [2.5, 1.0, 1.0, 0.5], [..."
Destination_Type,Type_of_Cab,Destination_Type,2.860000,0.414,3.0,"[[3.8461538461538463, 3.076923076923077, 1.538..."


In [19]:
PPM.Ch2Test_CatAssociation_over_yCat(cat_colsdf.index,label_col,df)

,label_col,colname,statistic,p_value,dof,expected_frequency
Surge_Pricing_Type,Surge_Pricing_Type,Surge_Pricing_Type,30.000000,0.000,4.0,"[[0.06666666666666667, 0.7333333333333333, 0.2..."
Customer_Since_Months,Surge_Pricing_Type,Customer_Since_Months,5.090909,0.405,5.0,"[[1.5714285714285714, 0.42857142857142855], [0..."
Gender,Surge_Pricing_Type,Gender,4.772727,0.092,2.0,"[[0.2, 2.2, 0.6], [0.8, 8.8, 2.4]]"
Cancellation_Last_1Month,Surge_Pricing_Type,Cancellation_Last_1Month,4.545455,0.603,6.0,"[[0.4666666666666667, 5.133333333333334, 1.4],..."
Type_of_Cab,Surge_Pricing_Type,Type_of_Cab,4.333333,0.632,6.0,"[[0.38461538461538464, 3.4615384615384617, 1.1..."
Confidence_Life_Style_Index,Surge_Pricing_Type,Confidence_Life_Style_Index,2.933333,0.569,4.0,"[[0.09090909090909091, 0.8181818181818182, 0.0..."
Destination_Type,Surge_Pricing_Type,Destination_Type,2.727273,0.842,6.0,"[[0.6666666666666666, 7.333333333333333, 2.0],..."


In [20]:
df.groupby(['Cancellation_Last_1Month'])['Type_of_Cab']

In [21]:
pd.crosstab(df['Customer_Since_Months'],df['Type_of_Cab'])

Type_of_Cab,A,B,C,D,E
Customer_Since_Months,,,,,
0.0,2970,3957,3548,2404,1454
1.0,2391,3313,2862,2023,1254
2.0,3180,4656,4092,2772,1747
3.0,2796,4078,3671,2481,1479
4.0,2229,3057,2782,1887,1134
5.0,2383,3462,2947,2100,1284
6.0,1997,2892,2628,1813,1102
7.0,1978,2883,2624,1841,1096
8.0,1677,2599,2346,1483,893


In [22]:
df.groupby(['Destination_Type'])['Type_of_Cab'].agg(pd.Series.mode)

Destination_Type
A    B
B    B
C    B
D    C
E    B
F    D
G    A
H    C
I    D
J    C
K    B
L    B
M    A
N    B
Name: Type_of_Cab, dtype: object

In [23]:
df['Customer_Since_Months'].describe()
df['Customer_Since_Months'].fillna(6.006048,inplace=True)

df['Life_Style_Index'].describe()
df['Life_Style_Index'].fillna(2.79,inplace=True)

In [24]:
CustSinceMonth_2CabType_MAP=df.groupby(['Customer_Since_Months'])['Type_of_Cab'].agg(pd.Series.mode)
df.loc[df['Type_of_Cab'].isnull(),'Type_of_Cab']=df.loc[df['Type_of_Cab'].isnull(),'Customer_Since_Months'].map(CustSinceMonth_2CabType_MAP)

In [25]:
df['Type_of_Cab'].isnull().sum()

0

In [26]:
df['Confidence_Life_Style_Index'].value_counts()

B    67265
C    59736
A    58536
Name: Confidence_Life_Style_Index, dtype: int64

In [27]:
PPM.Ch2Test_CatAssociation_over_yCat(cat_colsdf.index,'Confidence_Life_Style_Index',df)

,label_col,colname,statistic,p_value,dof,expected_frequency
Confidence_Life_Style_Index,Confidence_Life_Style_Index,Confidence_Life_Style_Index,22.000000,0.000,4.0,"[[0.09090909090909091, 0.45454545454545453, 0...."
Customer_Since_Months,Confidence_Life_Style_Index,Customer_Since_Months,16.720000,0.081,10.0,"[[0.18181818181818182, 0.9090909090909091, 0.9..."
Destination_Type,Confidence_Life_Style_Index,Destination_Type,7.542857,0.110,4.0,"[[0.6363636363636364, 3.1818181818181817, 3.18..."
Cancellation_Last_1Month,Confidence_Life_Style_Index,Cancellation_Last_1Month,5.866667,0.438,6.0,"[[0.45454545454545453, 2.272727272727273, 2.27..."
Gender,Confidence_Life_Style_Index,Gender,2.933333,0.231,2.0,"[[0.2727272727272727, 1.3636363636363635, 1.36..."
Surge_Pricing_Type,Confidence_Life_Style_Index,Surge_Pricing_Type,2.933333,0.569,4.0,"[[0.09090909090909091, 0.45454545454545453, 0...."
Type_of_Cab,Confidence_Life_Style_Index,Type_of_Cab,2.200000,0.900,6.0,"[[0.5454545454545454, 2.727272727272727, 2.727..."


In [28]:
CustSinceMonth_2Confidence_Life_Style_Index_MAP=df.groupby(['Customer_Since_Months'])['Confidence_Life_Style_Index'].agg(pd.Series.mode)
df.loc[df['Confidence_Life_Style_Index'].isnull(),'Confidence_Life_Style_Index']=df.loc[df['Confidence_Life_Style_Index'].isnull(),'Customer_Since_Months'].map(CustSinceMonth_2Confidence_Life_Style_Index_MAP)

In [29]:
df['Var1'].describe()
df['Var1'].fillna(61.0,inplace=True)

In [30]:
ContiBins=PPM.BiningContiCols_over_yBinary(['Var1','Var2','Var3','Customer_Since_Months','Customer_Rating','Trip_Distance','Life_Style_Index'],df,5)
ContiBins=ContiBins.astype('object')
df[ContiBins.columns]=ContiBins

In [31]:
PPM.desc1(df,2,label_col)

,dtypes,null_count,total count,unique_count,missing value ratio,variance of numerics,std,mean,min,max,skew,kurt,Oridinals SpearmanCorr withTarget
Surge_Pricing_Type,float64,87395.0,131662.0,3.0,39.9,0.54,0.74,2.16,1.00000,3.00,-0.255,-1.137,1.00000
Type_of_Cab,object,0.0,219057.0,5.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.51225
Cancellation_Last_1Month,int64,0.0,219057.0,9.0,0.0,1.07,1.04,0.78,0.00000,8.00,1.544,2.673,0.19586
Customer_Rating,float64,0.0,219057.0,3976.0,0.0,0.96,0.98,2.85,0.00125,5.00,-0.193,-0.537,0.14932
CR,int32,0.0,219057.0,6.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.14571
Customer_Rating_bins,object,0.0,219057.0,5.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.14292
Trip_Distance,float64,0.0,219057.0,10594.0,0.0,650.63,25.51,44.16,0.31000,109.23,0.726,-0.142,0.12111
Trip_Distance_bins,object,0.0,219057.0,5.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.11972
Confidence_Life_Style_Index,object,0.0,219057.0,3.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.11310
Life_Style_Index,float64,0.0,219057.0,69677.0,0.0,0.04,0.21,2.80,1.31785,4.88,0.251,1.888,0.07145


In [32]:
df.groupby("Type_of_Cab").size()

Type_of_Cab
A    35878
B    84953
C    46732
D    31885
E    19609
dtype: int64

In [33]:
df["Type_of_Cab"].value_counts()/ntrain

B    0.645236
C    0.354939
A    0.272501
D    0.242173
E    0.148934
Name: Type_of_Cab, dtype: float64

# frequency -or percentage Encoding
In count encoding we replace the categories by the count of the observations that show that category in the dataset. Similarly, we can replace the category by the frequency -or percentage- of observations in the dataset. That is, if 10 of our 100 observations show the colour blue, we would replace blue by 10 if doing count encoding, or by 0.1 if replacing by the frequency. These techniques capture the representation of each label in a dataset, but the encoding may not necessarily be predictive of the outcome. These are however, very popular encoding methods in Kaggle competitions.

In [34]:
['Var1','Var2','Var3','Customer_Rating','Trip_Distance','Life_Style_Index']
def FE_2Add_CatFrequencyCount3(_ntrain,_df,apply_or_test=0):
    print(""" Note: Its a count frequence encoding of each category
                # if categories have same frequency it can be an issue
                # will need to change it to ranked frequency encoding
                from scipy.stats import rankdata
          """)
    
    cat_temp=['Type_of_Cab','Cancellation_Last_1Month','CR','Life_Style_Index','Destination_Type','Customer_Since_Months','Gender']   
    
    _fe_train=pd.DataFrame()
    _fe_test=pd.DataFrame()
    for c in cat_temp:
        dic_train=dict(_df[:ntrain][c].value_counts()/_ntrain)
        _fe_train['FreqEncode_'+c]=_df[:ntrain][c].map(dic_train).astype('float32')
        dic_test=dict(_df[ntrain:][c].value_counts()/_ntrain)
        _fe_test['FreqEncode_'+c]=_df[ntrain:][c].map(dic_test).astype('float32')

    if apply_or_test==1:   
        _fe=_fe_train.append(_fe_test)
        _df=pd.concat([_df,_fe],1)
        return _df
    else:
        print(_fe_train.var().sort_values(ascending=False))
FE_2Add_CatFrequencyCount3(ntrain,df,apply_or_test=0)

 Note: Its a count frequence encoding of each category
                # if categories have same frequency it can be an issue
                # will need to change it to ranked frequency encoding
                from scipy.stats import rankdata
          
FreqEncode_Destination_Type            0.052769
FreqEncode_Gender                      0.037207
FreqEncode_Cancellation_Last_1Month    0.029914
FreqEncode_Customer_Since_Months       0.014880
FreqEncode_Type_of_Cab                 0.013045
FreqEncode_CR                          0.008697
FreqEncode_Life_Style_Index            0.003053
dtype: float32


In [35]:
df['CR'].dtypes

dtype('int32')

In [36]:
from sklearn.preprocessing import OneHotEncoder

# Create the encoder.
encoder = OneHotEncoder(handle_unknown="ignore")
encoder.fit(df[['CR','Type_of_Cab']])
encoder.transform(df[['CR','Type_of_Cab']])

<219057x11 sparse matrix of type '<class 'numpy.float64'>'
	with 438114 stored elements in Compressed Sparse Row format>

In [37]:
from scipy.stats import rankdata

In [38]:
pd.Series(rankdata(df['Type_of_Cab'])).var()

3694017125.313863

In [39]:
dict(df['Type_of_Cab'].value_counts()/df['Type_of_Cab'].shape[0])

{'B': 0.3878123045600004,
 'C': 0.21333260292983106,
 'A': 0.16378385534358636,
 'D': 0.1455557229396915,
 'E': 0.08951551422689072}

In [40]:
encoding = df.groupby('Type_of_Cab').size()
# get frequency of each category
encoding/len(df)

Type_of_Cab
A    0.163784
B    0.387812
C    0.213333
D    0.145556
E    0.089516
dtype: float64

In [41]:
df[label_col].value_counts()

2.0    56728
3.0    47720
1.0    27214
Name: Surge_Pricing_Type, dtype: int64

#  Target /mean encoding pros and cons
Target Encoding is also known as likelihood encoding or mean encoding. 


While mean encoding has shown to increase the quality of a classification model, it doesn’t go without its problems; the main one being the usual suspect, overfitting.
The fact that we are encoding the feature based on target classes may lead to data leakage, rendering the feature biased. To solve this, mean encoding is usually used with some type of Regularization. Check this solution on Kaggle as an example, where the author used an averaged cross-validation scheme.
And as long as we are talking about Kaggle, we might as well mention gradient boosting trees and how mean encoding is particularly useful with those. One of GBT’s downsides is its inability to handle high-cardinality categorical features, because trees have limited depth.

from Dmitry Altukhov’s “Concept of Mean Encoding”
Now, since mean encoding considerably decreases cardinality, as we’ve seen before, it becomes a great tool to use in order to reach a better loss with a shorter tree, and thus improving the classification model.

I've read a few articles on common encoding techniques, one of which caught my attention: mean encoding (aka target, aka likelihood, aka impact), whereby each distinct value of categorical value is replaced with average value of target variable we're trying to predict.

After some googling and reading through amazing Kaggle kernels and forums, I've realized that:

mean encoding as it is has high risk of overfitting, so some kind of regularisation is required
there are several ways of doing this regularisation
Dissecting their code and aplplying on single dataset led to this kernel.

In [42]:
# def FE_2Add_CatMeanOverContiAggregation2(_label_col,_df,apply=1):   
#     if apply==1:
#         cat_temp=['Type_of_Cab','CR','Destination_Type','Customer_Since_Months','Gender']
#         #conti_temp=['Var1','Var2','Var3','Customer_Rating','Trip_Distance','Life_Style_Index']
        
#         tempdata=_df[[_label_col]+cat_temp+conti_temp].copy()
#         _returnDF= pd.DataFrame()
#         for conti_colname in conti_temp:
#             print(conti_colname)
#             for cat_colname in cat_temp :
#                 for aggr in ['mean']:
#                     #print(conti_colname,cat_colname,aggr)
#                     tempAgg1=tempdata[tempdata[_label_col]==1].groupby(cat_colname)[conti_colname].agg(aggr)
#                     tempAgg2=tempdata[tempdata[_label_col]==2].groupby(cat_colname)[conti_colname].agg(aggr)
#                     tempAgg3=tempdata[tempdata[_label_col]==3].groupby(cat_colname)[conti_colname].agg(aggr)
#                     tempAgg12=tempAgg1/tempAgg2
#                     tempAgg23=tempAgg2/tempAgg3
#                     tempAgg123=1/tempAgg12* (1/tempAgg23)
#                     #tempdata[cat_colname+'0-'+aggr+'-over-'+conti_colname]=df_all[cat_colname].map(dict(tempAgg1))
#                     #tempdata[cat_colname+'1-'+aggr+'-over-'+conti_colname]=df_all[cat_colname].map(dict(tempAgg0))
#                     _returnDF[cat_colname+'1/0-'+aggr+'-over-'+conti_colname]=tempdata[cat_colname].map(dict(tempAgg123))
#     return _returnDF
#     #df_all=tempdata.copy()
#     #del tempdata
# #resultDF=FE_2Add_CatMeanOverContiAggregation2(label_col,df)
# #df[resultDF.columns]=resultDF
# #resultDF.isna().sum()
# #'Customer_Since_Months'


In [43]:
['Var1','Var2','Var3','Customer_Rating','Trip_Distance','Life_Style_Index']
def FE_2Add_CatMeanEncoding2(_label_Col,_ntrain,_df,apply_or_test=0):
    print(""" Note: Its a mean/target encoding of each category
               
          """)
    
    cat_temp=['Type_of_Cab','Cancellation_Last_1Month','CR','Life_Style_Index','Destination_Type','Customer_Since_Months','Gender']   
    
    _fe=pd.DataFrame()
    for c in cat_temp:
        dic_train1=_df[:ntrain].groupby([c]).agg({label_col:['mean']})
        dic_train1.columns=['target']
        dic_train=dict(dic_train1.target.round(3))
        #data_df = data_df.merge(mean_encoding,on=c,how='left')
        _fe['MeanEncode_'+c]=_df[c].map(dic_train)#.astype('float32')

    if apply_or_test==1:   
        _df=pd.concat([_df,_fe],1)
        return _df
    else:
        print(_fe.var().sort_values(ascending=False))
FE_2Add_CatMeanEncoding2(label_col,ntrain,df,apply_or_test=1)

 Note: Its a mean/target encoding of each category
               
          


,Cancellation_Last_1Month,Confidence_Life_Style_Index,Customer_Rating,Customer_Since_Months,Destination_Type,Gender,Life_Style_Index,Surge_Pricing_Type,Trip_Distance,Trip_ID,...,Customer_Rating_bins,Trip_Distance_bins,Life_Style_Index_bins,MeanEncode_Type_of_Cab,MeanEncode_Cancellation_Last_1Month,MeanEncode_CR,MeanEncode_Life_Style_Index,MeanEncode_Destination_Type,MeanEncode_Customer_Since_Months,MeanEncode_Gender
0,0,A,3.90500,1.000000,A,Female,2.42769,2.0,6.77,T0005689460,...,5,1,1,2.012,2.028,2.033,2.000,2.209,2.127,2.155
1,0,B,3.45000,10.000000,A,Male,2.78245,2.0,29.47,T0005689461,...,4,2,3,2.012,2.028,2.136,2.000,2.209,2.175,2.156
2,2,B,3.50125,10.000000,E,Male,2.79000,2.0,41.58,T0005689464,...,4,3,3,2.012,2.349,2.033,2.159,1.991,2.175,2.156
3,0,B,3.45375,10.000000,A,Male,2.79000,3.0,61.56,T0005689465,...,4,4,3,2.311,2.028,2.136,2.159,2.209,2.175,2.156
4,4,B,3.40250,10.000000,A,Male,3.03453,2.0,54.95,T0005689467,...,4,4,5,2.311,2.477,2.136,2.000,2.209,2.175,2.156
5,1,B,2.59750,10.000000,A,Male,2.79000,3.0,19.06,T0005689469,...,2,1,3,2.629,2.235,2.136,2.159,2.209,2.175,2.156
6,1,C,2.97500,10.000000,B,Male,2.83958,2.0,29.72,T0005689470,...,3,2,4,2.629,2.235,2.136,1.750,1.963,2.175,2.156
7,0,B,3.58250,2.000000,A,Male,2.81871,2.0,18.44,T0005689472,...,4,1,4,2.012,2.028,2.033,2.000,2.209,2.139,2.156
8,0,A,3.14625,3.000000,A,Male,2.79000,2.0,106.80,T0005689473,...,3,5,3,2.311,2.028,2.136,2.159,2.209,2.125,2.156
9,1,B,2.44375,5.000000,A,Male,3.04467,3.0,107.19,T0005689474,...,2,5,5,2.756,2.235,2.239,3.000,2.209,2.150,2.156


In [44]:
label_col

'Surge_Pricing_Type'

In [45]:
xxx=df[:ntrain].groupby(['Type_of_Cab']).agg({label_col:['mean']})
xxx.columns=['target']
dict(xxx.target.round(3))

{'A': 1.512, 'B': 2.012, 'C': 2.311, 'D': 2.756, 'E': 2.629}

# Weight of evidence

Weight of evidence (WOE) is a technique used to encode categorical variables for classification. WOE is the natural logarithm of the probability of the target being 1 divided the probability of the target being 0. WOE has the property that its value will be 0 if the phenomenon is random; it will be bigger than 0 if the probability of the target being 0 is bigger, and it will be smaller than 0 when the probability of the target being 1 is greater.

WOE transformation creates a nice visual representation of the variable, because by looking at the WOE encoded variable, we can see, category by category, whether it favours the outcome of 0, or of 1. In addition, WOE creates a monotonic relationship between variable and target, and leaves all the variables within the same value range.

In [46]:
cat_cols=list(df.columns[df.dtypes=='object'])
cat_cols#.pop(label_col)

['Confidence_Life_Style_Index',
 'Destination_Type',
 'Gender',
 'Trip_ID',
 'Type_of_Cab',
 'Var1_bins',
 'Var2_bins',
 'Var3_bins',
 'Customer_Since_Months_bins',
 'Customer_Rating_bins',
 'Trip_Distance_bins',
 'Life_Style_Index_bins']

In [47]:
# Commenting it --- As this is use when we want to Linear Model
def dummify(_df):
    cat_cols=list(_df.columns[_df.dtypes=='object'])
    print(cat_cols)
    if label_col in cat_cols:
        cat_cols.pop(label_col)
    
    print(_df.shape)
    _df[cat_cols]
    dummiedDF=pd.get_dummies(_df[cat_cols],cat_cols,drop_first=True)
    _df[dummiedDF.columns]=dummiedDF
    _df.drop(cat_cols,1,inplace=True)


    # Lets drop all the columns in entire data where testing data columns holding single value
    #### 4.1 Zero test for train and test data drop lowest variablity data
    zerotest_testing_col=[]
    for i in dummiedDF.columns:
        if i in _df.columns:
            zerotest_cond=_df[i][ntrain:].sum()==0
            if zerotest_cond:
                zerotest_testing_col.append(i)
    print(len(zerotest_testing_col),"-",zerotest_testing_col)
    if len(zerotest_testing_col) > 0:
       _df.drop(zerotest_testing_col,1,inplace=True)
    _df.shape

    # # Lets drop all the columns in entire data where training data columns holding single value
    zerotest_training_col=[]
    for i in dummiedDF.columns:
        if i in _df.columns:
            zerotest_cond=_df[i][:ntrain].sum()==0
            if zerotest_cond:
                zerotest_training_col.append(i)
    print(len(zerotest_training_col),"-",zerotest_training_col)
    if len(zerotest_training_col) > 0:
       _df.drop(zerotest_training_col,1,inplace=True)
    return _df


In [48]:
def statandardize(_label_col,_ntrain, _full_data,applyScale=''):
    if _label_col in _full_data.columns:
        _ytrain=_full_data[_label_col][:_ntrain]
        _full_data=_full_data.drop([_label_col],1)
    # Feature Scaling
    _ytrain = pd.Series([int(i) for i in _ytrain])
    _xtrain=_full_data[:ntrain]
    _xtest=_full_data[ntrain:]
    
    if applyScale =='std':
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        _xtrain = pd.DataFrame(sc.fit_transform(_xtrain),columns=_xtrain.columns)
        _xtest =  pd.DataFrame(sc.fit_transform(_xtest),columns=_xtest.columns)
    if applyScale =='minmax':
        from sklearn.preprocessing import MinMaxScaler
        sc = MinMaxScaler()
        _xtrain = pd.DataFrame(sc.fit_transform(_xtrain),columns=_xtrain.columns)
        _xtest =  pd.DataFrame(sc.fit_transform(_xtest),columns=_xtest.columns)

    return _xtrain,_ytrain,_xtest
#xtrain,ytrain,xtest=statandardize(label_col,ntrain,final_data,'') 

def bestRFE(ybinary,_xdata,k_value):
    from sklearn.feature_selection import RFE
    from sklearn.svm import SVR
    #estimator = SVR(kernel="linear")
    model_RFE = LogisticRegression()
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    model_lda1 = LinearDiscriminantAnalysis()
    rfe = RFE(estimator=model_lda1, n_features_to_select=k_value, step=2)
    fit= rfe.fit(_xdata, ybinary)
    ranking_df = pd.DataFrame()
    ranking_df['Feature_name'] = _xdata.columns[fit.n_features_]
    return _xdata.columns[fit.support_]

def executeToFindBest_RFE_cols_usingTrainSplit():
    xtrain,ytrain,xtest=statandardize(label_col,ntrain,final_data,'minmax')
    print(xtrain.shape)
    best_score=pd.DataFrame()
    for k in range(2,final_data.shape[1]-1,5):

        
        colList=bestRFE(ytrain,xtrain,k)
        #print("-------------------------",k,"---------------------------")
        custom_train=xtrain[colList]
        x_train, x_cv, y_train, y_cv = train_test_split(custom_train,ytrain, test_size =0.3,random_state=0,stratify=ytrain)
        # model fitting
        model_lr = LogisticRegression() 
        model_lr.fit(x_train, y_train)
        model_dt = DecisionTreeClassifier(random_state=45) 
        model_dt.fit(x_train, y_train)

        from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
        model_lda = LinearDiscriminantAnalysis()
        model_lda.fit(x_train, y_train)

        # Predict
        ptrain = model_lr.predict(x_train)
        ptest = model_lr.predict(x_cv)
        ptrain_dt = model_dt.predict(x_train)
        ptest_dt = model_dt.predict(x_cv)
        ptrain_lda = model_lda.predict(x_train)
        ptest_lda = model_lda.predict(x_cv)
        #1. Accuracy Score
        print("-----------RFE-----------" ,k, "--------- value-------------------")
        print("\n\n\n-----------RFE-----------" ,k, "--------- value-------------------\n")
        print("Logistic Regression:",accuracy_score(y_train,ptrain),accuracy_score(y_cv,ptest))
        print("LDA:",accuracy_score(y_train,ptrain_lda),accuracy_score(y_cv,ptest_lda))
        print("decisonTree:",accuracy_score(y_train,ptrain_dt),accuracy_score(y_cv,ptest_dt))
        best_score=best_score.append(pd.Series([k,'LogR',accuracy_score(y_train,ptrain),accuracy_score(y_cv,ptest)]),ignore_index=True)
        best_score=best_score.append(pd.Series([k,'LDA',accuracy_score(y_train,ptrain_lda),accuracy_score(y_cv,ptest_lda)]),ignore_index=True)
        best_score=best_score.append(pd.Series([k,'decisonTree',accuracy_score(y_train,ptrain_dt),accuracy_score(y_cv,ptest_dt)]),ignore_index=True)
    best_score.columns=['k-val','algo','train_accuracy','test_accuracy']
    return best_score.sort_values(['train_accuracy','test_accuracy'],ascending=[False,False]).head()
#executeToFindBest_RFE_cols_usingTrainSplit()

In [49]:
"""k-val	algo	train_accuracy	test_accuracy
44	30.0	decisonTree	1.000000	0.555558
47	32.0	decisonTree	1.000000	0.553153
41	28.0	decisonTree	0.999924	0.550394
32	22.0	decisonTree	0.996810	0.555280
35	24.0	decisonTree	0.996810	0.554141
https://www.coursera.org/lecture/python-machine-learning/multi-class-evaluation-1ugJR
"""

'k-val\talgo\ttrain_accuracy\ttest_accuracy\n44\t30.0\tdecisonTree\t1.000000\t0.555558\n47\t32.0\tdecisonTree\t1.000000\t0.553153\n41\t28.0\tdecisonTree\t0.999924\t0.550394\n32\t22.0\tdecisonTree\t0.996810\t0.555280\n35\t24.0\tdecisonTree\t0.996810\t0.554141\nhttps://www.coursera.org/lecture/python-machine-learning/multi-class-evaluation-1ugJR\n'

In [50]:
############################# Main #########################################
def executeToFindBest_RFE_cols_usingCV(_cv=5):
    def _classificationModelfit_CV(_algo,_xdata,_ydata,_cv):
        from sklearn.model_selection import cross_validate
        _algo.fit(_xdata, _ydata)
        from sklearn.metrics import confusion_matrix
        cv_matrix_score= cross_validate(_algo, _xdata, _ydata, cv=_cv,
                                        #scoring=('accuracy', 'precision', 'f1_weighted','recall','neg_log_loss'),
                                        return_train_score=True)
        avg_model_Scores=pd.DataFrame(cv_matrix_score).mean()
        for i in avg_model_Scores.index:
            if 'neg' in i :
                score="%s =%0.2f" %(i, avg_model_Scores[i]*-1) 
                #print(score)
            else:
                score="%s =%0.2f" %(i, avg_model_Scores[i] )
                #print(score)
        return avg_model_Scores
    
    def bestRFE(ybinary,_xdata,k_value):
        from sklearn.feature_selection import RFE
        #model_RFE = LogisticRegression()
        from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
        model_lda = LinearDiscriminantAnalysis()
        rfe = RFE(estimator=model_lda, n_features_to_select=k_value, step=1)
        fit= rfe.fit(xtrain, ytrain)
        ranking_df = pd.DataFrame()
        ranking_df['Feature_name'] = xtrain.columns[fit.n_features_]
        return xtrain.columns[fit.support_]
    
    #####################   Main Program Decision tree ##############
    print(final_data.shape)
    best_score=pd.DataFrame()
    #for k in range(2,final_data.shape[1],2):
    xtrain,ytrain,xtest=statandardize(label_col,ntrain,final_data,'')
    for k in range(2,final_data.shape[1]-1,5):
        
        colList=bestRFE(ytrain,xtrain,k)
        custom_train=xtrain[colList]
        print("-----------------------------",k,"---------------------------")

#         model_lr = LinearDiscriminantAnalysis()
#         ser=_classificationModelfit_CV(model_lr, custom_train,ytrain,_cv)
#         ser['Algo']='LogR'
#         ser['RFE_Col_len']=k
#         best_score=best_score.append(ser,ignore_index=True)
        

        model_dt = DecisionTreeClassifier(random_state=45) 
        ser=_classificationModelfit_CV(model_dt, custom_train,ytrain,_cv)
        ser['Algo']='DT'
        ser['RFE_Col_len']=k
        best_score=best_score.append(ser,ignore_index=True)
        
        from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
        model_lda = LinearDiscriminantAnalysis()        
        ser=_classificationModelfit_CV(model_lda, custom_train,ytrain,_cv)
        ser['Algo']='LDA'
        ser['RFE_Col_len']=k
        best_score=best_score.append(ser,ignore_index=True)

    return best_score#.sort_values(['test_accuracy','train_accuracy'],ascending=[False,False]).head()


In [51]:
#DT_CV=executeToFindBest_RFE_cols_usingCV(_cv=5)

In [52]:
#DT_CV.sort_values(['train_score','test_score'],ascending=[False,False])

In [53]:
def generate_submission_file(submission_csv_name,model,org_test,cleaned_test):
    y_pred = model.predict(cleaned_test)

    #Export submission file:
    org_test[label_col]=[int(i) for i in y_pred]
    submission=org_test[['Trip_ID',label_col]]
    submission[label_col]= submission[label_col]#.map({1:"Y",0:"N"})
#     negRec=len(submission[submission.Item_Outlet_Sales<0])
#     if negRec >0:
#         print("Output contain Negative records:", negRec)
#         submission.to_csv(submission_csv_name, index=False)
#         return submission[submission.Item_Outlet_Sales<0]
#     else:
    print("saved File :",submission_csv_name)
    submission.to_csv(submission_csv_name, index=False)
        
    return submission[label_col]


def classificationModelfit_CV2(_algo,_xdata,_ydata,_cv):
    from sklearn.model_selection import cross_validate
    _algo.fit(_xdata, _ydata)
    #Perform cross-validation:
    from sklearn.metrics import confusion_matrix
    def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
    def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
    def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
    def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]
#     scoring1 = {'tp': make_scorer(tp), 'tn': make_scorer(tn),
#                 'fp': make_scorer(fp), 'fn': make_scorer(fn)}
    cv_matrix_score= cross_validate(_algo, _xdata, _ydata, cv=_cv,
                                    #scoring=('accuracy', 'precision','f1', 'recall','roc_auc','neg_log_loss',),
                         return_train_score=True)
    avg_model_Scores=pd.DataFrame(cv_matrix_score).mean()
    for i in avg_model_Scores.index:
        if 'neg' in i :
            score="%s =%0.3f" %(i, avg_model_Scores[i]*-1) 
            #_df.loc[i]=np.sqrt(avg_model_Scores[i]*-1)
            print(score)
        else:
            score="%s =%0.3f" %(i, avg_model_Scores[i] )
            #_df.loc[i]=
            #avg_model_Scores[i]
            print(score)
    return avg_model_Scores




def bestRFE(ybinary,_xdata,k_value):
    from sklearn.feature_selection import RFE
    from sklearn.svm import SVR
    from sklearn import svm
    #estimator = SVR(kernel="linear")
    #model_RFE = LogisticRegression()
    svc = svm.SVC(kernel='linear', C=1,gamma=0.1)
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    model_lda1 = LinearDiscriminantAnalysis()
    rfe = RFE(estimator=model_lda1, n_features_to_select=k_value, step=2)
    fit= rfe.fit(_xdata, ybinary)
    ranking_df = pd.DataFrame()
    ranking_df['Feature_name'] = _xdata.columns[fit.n_features_]
    return _xdata.columns[fit.support_]



In [54]:
# print(xtrain.shape)
# xtrain,ytrain,xtest=statandardize(label_col,ntrain,final_data,'minmax')
# RFE_cols=bestRFE(ytrain,xtrain,50)
# #print(RFE_cols)
# model_dt = DecisionTreeClassifier(random_state=70)#,max_depth=3,min_samples_split=2,min_samples_leaf=3)
# d=classificationModelfit_CV2(model_dt, xtrain[RFE_cols],ytrain,5)
# xGBC_pred=generate_submission_file("n2.csv",model_dt,test,xtest[RFE_cols])

In [55]:
#xGBC_pred.value_counts()

In [56]:
def DecisionTreeParamTuning(ybinary,_xdata,_cv,max_depth=[4,5],min_samples_split=[2,3,4]):
    def _classificationModelfit_CV(_algo,_xdata,_ydata,_cv):
        from sklearn.model_selection import cross_validate
        _algo.fit(_xdata, _ydata)
        from sklearn.metrics import confusion_matrix
        cv_matrix_score= cross_validate(_algo, _xdata, _ydata, cv=_cv,
                                        #scoring=('accuracy', 'precision', 'f1','recall', 'roc_auc','neg_log_loss'),
                                        return_train_score=True)
        avg_model_Scores=pd.DataFrame(cv_matrix_score).mean()
        for i in avg_model_Scores.index:
            if 'neg' in i :
                score="%s =%0.2f" %(i, avg_model_Scores[i]*-1) 
                #print(score)
            else:
                score="%s =%0.2f" %(i, avg_model_Scores[i] )
                #print(score)
        return avg_model_Scores

    combi_code=0
    combi=[]
    _final_df=pd.DataFrame()
    for depth in max_depth:
        for samp_split in min_samples_split:
            combi_code=combi_code+1
            combi.append(str([depth,samp_split]))
            model_dt = DecisionTreeClassifier(random_state=22,max_depth=depth,min_samples_split=samp_split)
            result=_classificationModelfit_CV(model_dt, _xdata,ybinary,_cv)
            result['Combination_code_param']=combi_code
            result['param_combination']=str([depth,samp_split])
            _final_df=_final_df.append(pd.DataFrame(result).T)
            #_final_df
    #_final=_final_df[['test_accuracy','train_accuracy','test_f1','train_f1','test_roc_auc','train_roc_auc','Combination_code_param']].melt("Combination_code_param",var_name='Metrics',value_name="Values")
    #print(np.array(_final_df[['Combination_code_param','param_combination']]))
    #plt.figure(figsize=(20,6))
    #sns.pointplot(x="Combination_code_param", y="Values",hue="Metrics", data=_final)
    
    #plt.xticks([i for i in range(1, combi_code+1)],combi,fontsize=15,rotation=80)
    #plt.yticks(fontsize=15)
    #plt.xlabel("param combination",fontsize=15)
    #plt.ylabel("score value",fontsize=15)
    #plt.legend(fontsize=20)
    return _final_df       

In [57]:
# xtrain,ytrain,xtest=statandardize(label_col,ntrain,final_data,'minmax')
# RFE_cols=bestRFE(ytrain,xtrain,28)
#dt_score=DecisionTreeParamTuning(ytrain,xtrain[RFE_cols],5,max_depth=[3,6,8,13,15,18,20,22],min_samples_split=[2,3,4,5,10,15,25,40])
#dt_score.sort_values(['train_score','test_score'],ascending=[False,False]).head(3)

In [58]:
#dt_score.sort_values(['test_score','train_score'],ascending=[False,False]).head()

In [59]:
#xtrain,ytrain,xtest=statandardize(label_col,ntrain,final_data,'minmax')
#RFE_cols=bestRFE(ytrain,xtrain,28)
#print(RFE_cols)
#model_dt = DecisionTreeClassifier(random_state=70,max_depth=6,min_samples_split=2)
#d=classificationModelfit_CV2(model_dt, xtrain[RFE_cols],ytrain,5)
#xGBC_pred=generate_submission_file("n3.csv",model_dt,test,xtest[RFE_cols])

In [60]:
# from sklearn.ensemble import RandomForestClassifier
# rf=RandomForestClassifier(random_state=45,criterion='entropy',max_depth=9,min_samples_split=2,n_estimators=100)
# d=classificationModelfit_CV2(rf, xtrain[RFE_cols],ytrain,5)

In [61]:
# from sklearn.ensemble import RandomForestClassifier
# rf=RandomForestClassifier(random_state=45,criterion='entropy',max_depth=15,min_samples_split=2,n_estimators=100)
# d=classificationModelfit_CV2(rf, xtrain[RFE_cols],ytrain,5)
# xGBC_pred=generate_submission_file("n2.csv",rf,test,xtest[RFE_cols])

In [62]:
# from sklearn.ensemble import RandomForestClassifier
# rf=RandomForestClassifier(random_state=45,criterion='entropy',
#                           max_depth=15,min_samples_split=4,
#                           max_leaf_nodes=800,n_estimators=100)
# d=classificationModelfit_CV2(rf, xtrain[RFE_cols],ytrain,5)
# xGBC_pred=generate_submission_file("n2.csv",rf,test,xtest[RFE_cols])

In [63]:
PPM.desc1(df,2,label_col)

,dtypes,null_count,total count,unique_count,missing value ratio,variance of numerics,std,mean,min,max,skew,kurt,Oridinals SpearmanCorr withTarget
Surge_Pricing_Type,float64,87395.0,131662.0,3.0,39.9,0.54,0.74,2.16,1.00000,3.00,-0.255,-1.137,1.00000
Type_of_Cab,object,0.0,219057.0,5.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.51225
Cancellation_Last_1Month,int64,0.0,219057.0,9.0,0.0,1.07,1.04,0.78,0.00000,8.00,1.544,2.673,0.19586
Customer_Rating,float64,0.0,219057.0,3976.0,0.0,0.96,0.98,2.85,0.00125,5.00,-0.193,-0.537,0.14932
CR,int32,0.0,219057.0,6.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.14571
Customer_Rating_bins,object,0.0,219057.0,5.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.14292
Trip_Distance,float64,0.0,219057.0,10594.0,0.0,650.63,25.51,44.16,0.31000,109.23,0.726,-0.142,0.12111
Trip_Distance_bins,object,0.0,219057.0,5.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.11972
Confidence_Life_Style_Index,object,0.0,219057.0,3.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.11310
Life_Style_Index,float64,0.0,219057.0,69677.0,0.0,0.04,0.21,2.80,1.31785,4.88,0.251,1.888,0.07145


In [64]:
-0.164<-0.08

True

In [65]:
tempdf=df.copy()

In [66]:
def ContiTransformation(_df):
    conti_cols_FE=['Var2','Var1', 'Var3','Life_Style_Index','Trip_Distance','Customer_Rating','Customer_Since_Months_X_Customer_Rating']
    NORMQUINT_TRANS=PPM.get_conti_cols_transformed(conti_cols_FE[1:],_df,e=1,type='NQ')
    _df[NORMQUINT_TRANS.columns]=np.array(NORMQUINT_TRANS) 
    return _df


In [67]:
pd.crosstab(tempdf['Confidence_Life_Style_Index'],tempdf['Gender'])

Gender,Female,Male
Confidence_Life_Style_Index,,
A,20345,50128
B,24758,61530
C,17826,44470


In [68]:
cond=[ (df['Confidence_Life_Style_Index']=='A')&(df['Gender'] =='Male'),
       (df['Confidence_Life_Style_Index']=='B')&(df['Gender'] =='Male'),
      (df['Confidence_Life_Style_Index']=='C')&(df['Gender'] =='Male'),
      (df['Confidence_Life_Style_Index']=='C')&(df['Gender'] =='Female'),
     ]

tempdf['GenderWise_Confidence_Life_Style_Index']=np.select(cond,[1,2,1,4],default=3)
tempdf['GenderWise_Confidence_Life_Style_Index_OT']=tempdf['GenderWise_Confidence_Life_Style_Index'].astype('object')
tempdf['GenderWise_Confidence_Life_Style_Index_OT'].isna().sum()

0

In [69]:
def customFE(_df):
    cond=[(_df['Gender']=='Male')& (_df['Cancellation_Last_1Month']==0), # 4
          (_df['Gender']=='Female')& (_df['Cancellation_Last_1Month']==0), # 3
          (_df['Gender']=='Male')& (_df['Cancellation_Last_1Month'].isin([1])), # 3
          (_df['Gender']=='Female')& (_df['Cancellation_Last_1Month'].isin([1,2])), # 2
          (_df['Gender']=='Male')& (_df['Cancellation_Last_1Month'].isin([2,3])), # 2
         ]
    choices=[4,3,3,2,2]
    np.select(cond,choices,default=1)
    _df['GenderWise_Cancellation_Last_1Month']=np.select(cond,choices,default=1)
    _df['GenderWise_Cancellation_Last_1Month_OT']=_df['GenderWise_Cancellation_Last_1Month'].astype('object')
    print(_df['GenderWise_Cancellation_Last_1Month'].isna().sum())
    
    
    cond=[ (_df['Confidence_Life_Style_Index']=='A')&(_df['Gender'] =='Male'),
       (_df['Confidence_Life_Style_Index']=='B')&(_df['Gender'] =='Male'),
      (_df['Confidence_Life_Style_Index']=='C')&(_df['Gender'] =='Male'),
      (_df['Confidence_Life_Style_Index']=='C')&(_df['Gender'] =='Female'),
     ]

    _df['GenderWise_Confidence_Life_Style_Index']=np.select(cond,[1,2,1,4],default=3)
    _df['GenderWise_Confidence_Life_Style_Index_OT']=_df['GenderWise_Confidence_Life_Style_Index'].astype('object')
    _df['GenderWise_Confidence_Life_Style_Index_OT']

    return _df
    

In [83]:
final_data=df.copy()
final_data=customFE(final_data)
final_data['Customer_Since_Months_X_Customer_Rating']=final_data['Customer_Since_Months']*final_data['Customer_Rating']
final_data['CRWiseMeanTripDistance']=final_data['CR'].map(df.groupby('CR')['Trip_Distance'].mean().astype('int32'))
final_data=ContiTransformation(final_data)
if 'Trip_ID' in final_data.columns:
    final_data.drop('Trip_ID',1,inplace=True)
 
final_data=FE_2Add_CatFrequencyCount3(ntrain,final_data,apply_or_test=1)
#print(final_data.isna().sum())
#resultDF=FE_2Add_CatMeanOverContiAggregation2(label_col,final_data)
final_data=FE_2Add_CatMeanEncoding2(label_col,ntrain,final_data,apply_or_test=1)
print("Trip_ID exist:",'Trip_ID' in final_data.columns)
#final_data.head()
summy=PPM.desc1(final_data,2,label_col)
#c=
summy[summy['dtypes']=='object'].index

0
Var1
Var3
Life_Style_Index
Trip_Distance
Customer_Rating
Customer_Since_Months_X_Customer_Rating
 Note: Its a count frequence encoding of each category
                # if categories have same frequency it can be an issue
                # will need to change it to ranked frequency encoding
                from scipy.stats import rankdata
          
 Note: Its a mean/target encoding of each category
               
          
Trip_ID exist: False


Index(['Type_of_Cab', 'GenderWise_Cancellation_Last_1Month_OT',
       'Customer_Rating_bins', 'Trip_Distance_bins',
       'Confidence_Life_Style_Index',
       'GenderWise_Confidence_Life_Style_Index_OT', 'Life_Style_Index_bins',
       'Destination_Type', 'Var3_bins', 'Customer_Since_Months_bins',
       'Var1_bins', 'Var2_bins', 'Gender'],
      dtype='object')

In [84]:
summy

,dtypes,null_count,total count,unique_count,missing value ratio,variance of numerics,std,mean,min,max,skew,kurt,Oridinals SpearmanCorr withTarget
Surge_Pricing_Type,float64,87395.0,131662.0,3.0,39.9,0.54,0.74,2.16,1.00000,3.00,-0.255,-1.137,1.00000
MeanEncode_Life_Style_Index,float64,18406.0,200651.0,62.0,8.4,0.24,0.49,2.16,1.00000,3.00,-0.286,0.304,0.62507
MeanEncode_Type_of_Cab,float64,0.0,219057.0,5.0,0.0,0.15,0.39,2.16,1.51200,2.76,-0.064,-0.801,0.52039
Type_of_Cab,object,0.0,219057.0,5.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.51225
FreqEncode_Type_of_Cab,float32,0.0,219057.0,10.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.25134
MeanEncode_Cancellation_Last_1Month,float64,0.0,219057.0,9.0,0.0,0.02,0.14,2.16,2.02800,3.00,0.535,-1.103,0.19589
Cancellation_Last_1Month,int64,0.0,219057.0,9.0,0.0,1.07,1.04,0.78,0.00000,8.00,1.544,2.673,0.19586
FreqEncode_Cancellation_Last_1Month,float32,0.0,219057.0,18.0,0.0,0.00,0.00,0.00,0.00000,0.00,0.000,0.000,0.19586
MeanEncode_Destination_Type,float64,0.0,219057.0,14.0,0.0,0.02,0.13,2.16,1.76500,2.45,-0.316,-0.468,0.17248
GenderWise_Cancellation_Last_1Month,int64,0.0,219057.0,4.0,0.0,0.76,0.87,3.06,1.00000,4.00,-0.442,-0.819,0.16918


In [85]:
fdata=dummify(final_data)


['Confidence_Life_Style_Index', 'Destination_Type', 'Gender', 'Type_of_Cab', 'Var1_bins', 'Var2_bins', 'Var3_bins', 'Customer_Since_Months_bins', 'Customer_Rating_bins', 'Trip_Distance_bins', 'Life_Style_Index_bins', 'GenderWise_Cancellation_Last_1Month_OT', 'GenderWise_Confidence_Life_Style_Index_OT']
(219057, 41)
0 - []
0 - []


In [86]:
# xtrain,ytrain,xtest=statandardize(label_col,ntrain,fdata,'')
# RFE_cols=bestRFE(ytrain,xtrain,int(xtrain.shape[1]*0.85))
# print(RFE_cols)
# model_dt = DecisionTreeClassifier(random_state=70,max_depth=9,min_samples_split=2,min_samples_leaf=3)
# #d=classificationModelfit_CV2(model_dt, xtrain[RFE_cols],ytrain,5)
# d=classificationModelfit_CV2(model_dt, xtrain,ytrain,5)

# #xGBC_pred=generate_submission_file("n2.csv",model_dt,test,xtest[RFE_cols])

In [87]:
#executeToFindBest_RFE_cols_usingTrainSplit()

In [88]:
xtrain,ytrain,xtest=statandardize(label_col,ntrain,fdata,'minmax')
RFE_cols=bestRFE(ytrain,xtrain,int(xtrain.shape[1]*0.85))
#RFE_cols=bestRFE(ytrain,xtrain,32)
#print(RFE_cols)
#model_dt = DecisionTreeClassifier(random_state=70,max_depth=9,min_samples_split=2,min_samples_leaf=3)
#d=classificationModelfit_CV2(model_dt, xtrain[RFE_cols],ytrain,5)
#d=classificationModelfit_CV2(model_dt, xtrain,ytrain,5)

# 
#### XGBoost softprob or softmax.
param <- list("objective" = "multi:softprob",    # multiclass classification 
              "num_class" = 12,    # number of classes 
              "eval_metric" = "merror",    # evaluation metric 
              "nthread" = 6,   # number of threads to be used 
              "max_depth" = 15,    # maximum depth of tree 
              "eta" = 0.07,    # step size shrinkage 
              "subsample" = 0.8,    # part of data instances to grow tree 
              "colsample_bytree" = 0.9  # subsample ratio of columns when constructing each tree 
              
)

Basically both softmax and softprob are used for multiclass classification. It’s the output which separates them. In Softmax you will get the class with the maximum probability as output, but with Softprob you will get a matrix with probability value of each class you are trying to predict.


In [96]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [97]:
# Lets me reduce the dataset
#(pd.Series(ytrain).value_counts()* .35)
from imblearn.datasets import make_imbalance
X, y = make_imbalance(xtrain, ytrain,
                      sampling_strategy={1: 9524, 2: 19854, 3: 16702},
                      random_state=45)

In [98]:
X=pd.DataFrame(X,columns=xtrain.columns)

In [100]:
from xgboost import XGBClassifier
def XGBClassifierParamTuning(ybinary,_xdata,_cv,learning_rate=[.1,.08,0.05,0.03,.01],n_estimators=[50,100]):
    def _classificationModelfit_CV(_algo,_xdata,_ydata,_cv):
        from sklearn.model_selection import cross_validate
        _algo.fit(_xdata, _ydata)
        from sklearn.metrics import confusion_matrix
        cv_matrix_score= cross_validate(_algo, _xdata, _ydata, cv=_cv,
                                        #scoring=('accuracy', 'precision', 'f1','recall', 'roc_auc','neg_log_loss'),
                                        return_train_score=True)
        avg_model_Scores=pd.DataFrame(cv_matrix_score).mean()
        for i in avg_model_Scores.index:
            if 'neg' in i :
                score="%s =%0.2f" %(i, avg_model_Scores[i]*-1) 
                #print(score)
            else:
                score="%s =%0.2f" %(i, avg_model_Scores[i] )
                #print(score)
        return avg_model_Scores

    combi_code=0
    combi=[]
    _final_df=pd.DataFrame()
    for _lr in learning_rate:
        for n_est in n_estimators:
            combi_code=combi_code+1
            combi.append(str([_lr,n_est]))
            xgb1 = XGBClassifier(learning_rate =_lr, n_estimators=n_est, max_depth=9, min_child_weight=2, gamma=0, subsample=0.8,
                     colsample_bytree=0.8, objective= 'multi:softmax', 
                                 num_class = 3,
                                 nthread=4, scale_pos_weight=1, seed=27)
            result=_classificationModelfit_CV(xgb1, _xdata,ybinary,_cv)
            result['Combination_code_param']=combi_code
            result['param_combination']=str([_lr,n_est])
            print(result)
            _final_df=_final_df.append(pd.DataFrame(result).T)
            _final_df
#     _final=_final_df[['test_accuracy','train_accuracy','test_f1','train_f1','test_roc_auc','train_roc_auc','Combination_code_param']].melt("Combination_code_param",var_name='Metrics',value_name="Values")
#     #print(np.array(_final_df[['Combination_code_param','param_combination']]))
#     plt.figure(figsize=(20,6))
#     sns.pointplot(x="Combination_code_param", y="Values",hue="Metrics", data=_final)
    
#     plt.xticks([i for i in range(1, combi_code+1)],combi,fontsize=15,rotation=80)
#     plt.yticks(fontsize=15)
#     plt.xlabel("param combination",fontsize=15)
#     plt.ylabel("score value",fontsize=15)
#     plt.legend(fontsize=20)
    return _final_df       
#xgb_score=XGBClassifierParamTuning(y,X[RFE_cols],5)

In [102]:
# [.1,.08,0.05,0.03,.01]
from xgboost import XGBClassifier
xgb1 = XGBClassifier(learning_rate =0.05, n_estimators=300, max_depth=7, min_child_weight=3, subsample=0.8,
                     colsample_bytree=0.8, objective= 'multi:softmax', 
                                 num_class = 3, nthread=4, seed=27, gamma=2,scale_pos_weight=.72)
                    # ,reg_alpha=.0003,reg_lambda=.008)
#modelfit(xgb1, train, predictors)
d=classificationModelfit_CV2(xgb1,X,y,5)
xGBC_pred=generate_submission_file("n1.csv",xgb1,test,xtest)
# # SCORING 0.8194
# # Approach to Tune XGB
# #1. roughly decide learning rate [0.1<> 0.01] 
# #2. find the best estimators
# #3. find the depth ... i have taken best max depth tunned during decision tree tuning
# #4. find the best gamma
# #5. find the best scale_pos_weight
# #6. find the best reg_alpha

fit_time =121.111
score_time =0.649
test_score =0.792
train_score =0.838
saved File : n1.csv
